In [1]:
# Cell 1: Load and Combine Normalized Data

import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm

# --- Path Setup ---
ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent

NORMALIZED_DIR = ROOT / "data" / "processed" / "tmp_normalized"

# --- Load and Combine Data Chunks ---
all_files = sorted(NORMALIZED_DIR.glob("normalized_chunk_*.csv"))

if not all_files:
    print(f"❌ Error: No normalized data files found in '{NORMALIZED_DIR}'.")
    print("Please run the '02_enrich_and_normalize.ipynb' notebook first.")
else:
    print(f"Found {len(all_files)} normalized data chunks. Combining them now...")
    
    # Read each chunk and append it to a list
    df_list = [pd.read_csv(f) for f in tqdm(all_files, desc="Loading chunks")]
    
    # Concatenate all DataFrames in the list into one master DataFrame
    df = pd.concat(df_list, ignore_index=True)
    
    # --- Verification ---
    print("\n✅ Master DataFrame created successfully.")
    print(f"Total rows: {len(df):,}")
    
    print("\nDataFrame Info:")
    df.info()
    
    print("\nSample of the combined data:")
    display(df.head())

Found 58 normalized data chunks. Combining them now...


Loading chunks:   0%|          | 0/58 [00:00<?, ?it/s]


✅ Master DataFrame created successfully.
Total rows: 1,126,844

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126844 entries, 0 to 1126843
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   qid         1126844 non-null  object
 1   title       1125590 non-null  object
 2   gender      1126844 non-null  object
 3   country     1126844 non-null  object
 4   occupation  1126844 non-null  object
dtypes: object(5)
memory usage: 43.0+ MB

Sample of the combined data:


,qid,title,gender,country,occupation
0,Q1000505,Bud Lee (pornographer),male,United States,film director
1,Q1000682,Fernando Carrillo,male,Venezuela,singer
2,Q1001324,Buddy Rice,male,United States,racing automobile driver
3,Q1004037,Frederik X,male,Kingdom of Denmark,aristocrat
4,Q100520438,1984 New York City Subway shooting,unknown,unknown,unknown


In [2]:
# Cell 2: Merge with Creation Timestamps

print("Loading the seed file with creation timestamps...")

try:
    # Find the most recent seed file in the 'data/raw' directory
    seed_path = sorted((ROOT / "data" / "raw").glob("seed_enwiki_*.csv"))[-1]
    seed_df = pd.read_csv(seed_path)
    print(f"✅ Loaded seed file: {seed_path.name}")
    
    # Merge the timestamp data into our main DataFrame using 'qid' as the key
    # We only need the 'qid' and 'first_edit_ts' columns for the merge
    df = pd.merge(
        df,
        seed_df[['qid', 'first_edit_ts']],
        on='qid',
        how='left'
    )
    
    # Convert the timestamp string into a proper datetime object for analysis
    # The 'Z' at the end of the string correctly tells pandas it's in UTC
    df['first_edit_ts'] = pd.to_datetime(df['first_edit_ts'])
    
    # --- Verification ---
    print("\n✅ Timestamps merged successfully.")
    print("\nUpdated DataFrame Info:")
    df.info()
    
    print("\nSample of the data with timestamps:")
    display(df.head())

except IndexError:
    print("❌ Error: No seed file found in 'data/raw/'.")
    print("This file is the final output of '01_api_seed.ipynb'. Please run it first.")

Loading the seed file with creation timestamps...
✅ Loaded seed file: seed_enwiki_20251007-213232.csv

✅ Timestamps merged successfully.

Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126844 entries, 0 to 1126843
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype              
---  ------         --------------    -----              
 0   qid            1126844 non-null  object             
 1   title          1125590 non-null  object             
 2   gender         1126844 non-null  object             
 3   country        1126844 non-null  object             
 4   occupation     1126844 non-null  object             
 5   first_edit_ts  1126148 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 51.6+ MB

Sample of the data with timestamps:


,qid,title,gender,country,occupation,first_edit_ts
0,Q1000505,Bud Lee (pornographer),male,United States,film director,2004-02-08 20:34:03+00:00
1,Q1000682,Fernando Carrillo,male,Venezuela,singer,2003-05-25 02:28:18+00:00
2,Q1001324,Buddy Rice,male,United States,racing automobile driver,2004-05-31 07:37:12+00:00
3,Q1004037,Frederik X,male,Kingdom of Denmark,aristocrat,2003-10-12 03:02:54+00:00
4,Q100520438,1984 New York City Subway shooting,unknown,unknown,unknown,2003-08-06 05:08:33+00:00


In [3]:
# Cell 3: Occupation Bucketing 

# Comprehensive version of the bucketing logic to ensure the 'Other' category is minimized.

print("Defining final occupation buckets...")

# 1. Define the most comprehensive categories.
OCCUPATION_BUCKETS = {
    "Sports": [
        "association football player", "american football player", "basketball player", "cricketer", "athletics competitor", 
        "ice hockey player", "baseball player", "rugby union player", "sport cyclist", "swimmer", "racing automobile driver", 
        "coach", "boxer", "athlete", "tennis player", "rower", "australian rules football player", "rugby league player", 
        "handball player", "volleyball player", "judoka", "racing driver", "golfer", "chess player", "badminton player", 
        "sprinter", "figure skater", "sport shooter", "weightlifter", "fencer", "artistic gymnast", "curler", 
        "mixed martial arts fighter", "professional wrestler", "water polo player", "association football manager", 
        "basketball coach", "amateur wrestler", "field hockey player", "canoeist", "alpine skier", "sailor", 
        "canadian football player", "cross-country skier", "motorcycle racer", "biathlete", "table tennis player", 
        "speed skater", "hurler", "rhythmic gymnast", "gaelic football player", "archer", "taekwondo athlete", 
        "competitive diver", "long-distance runner", "equestrian", "ski jumper", "squash player", "head coach", 
        "association football referee", "marathon runner", "freestyle skier", "bobsledder", "snowboarder", "gymnast", 
        "luger", "triathlete", "bowls player", "poker player", "middle-distance runner", "kayaker", "darts player", 
        "karateka", "sports commentator", "ice dancer", "softball player", "snooker player", "jockey", "kickboxer", 
        "orienteer", "modern pentathlete", "speedway rider", "short-track speed skater", "lacrosse player", 
        "synchronized swimmer", "netballer", "rikishi", "track cyclist", "thai boxer", "professional gamer", 
        "american football coach", "rally driver", "beach volleyball player", "mountaineer", "sports executive", 
        "professional baseball player", "nordic combined skier", "javelin thrower", "surfer", "skateboarder", 
        "hurdler", "para swimmer", "coxswain", "powerlifter", "para athletics competitor", "dressage rider", 
        "skeleton racer", "skipper", "horse trainer", "futsal player", "pole vaulter", "bodybuilder", 
        "rugby sevens player", "bridge player", "trampoline gymnast", "pool player", "martial artist", "racewalker", 
        "bowler", "high jumper", "show jumper", "ice hockey coach", "wheelchair curler", "motocross rider", 
        "windsurfer", "go professional", "long jumper", "rock climber", "ski mountaineer", "paralympic athlete", 
        "handball coach", "cyclo-cross cyclist", "hammer thrower", "acrobatic gymnast", "para badminton player", 
        "para table tennis player", "shot putter", "wheelchair tennis player", "formula one driver", "referee", 
        "rugby union coach", "baseball umpire", "ultramarathon runner", "kabaddi player", "discus thrower", 
        "wrestler", "event rider", "nascar team owner", "bandy player", "skier", "runner", "triple jumper", 
        "softball coach", "cricket umpire", "sitting volleyball player", "steeplechase runner", "tennis coach", 
        "professional golfer", "standing volleyball player", "magic: the gathering player", "rugby player", 
        "polo player", "boccia player"
    ],
    "Politics & Law": [
        "politician", "lawyer", "judge", "diplomat", "civil servant", "activist", "human rights activist", 
        "jurist", "police officer", "trade unionist", "legal scholar", "lgbtq rights activist", "official", 
        "barrister", "political activist", "women's rights activist", "lobbyist", "aristocrat", "justice of the peace", 
        "member of the state duma", "political adviser", "magistrate", "peace activist", "social activist", 
        "statesperson", "spy", "climate activist"
    ],
    "Arts & Culture": [
        "actor", "writer", "singer", "journalist", "film director", "musician", "artist", "photographer", 
        "painter", "poet", "rapper", "composer", "screenwriter", "record producer", "model", "comedian", 
        "television presenter", "singer-songwriter", "songwriter", "film producer", "television actor", 
        "opera singer", "jazz musician", "pianist", "sculptor", "guitarist", "conductor", "stage actor", 
        "radio personality", "disc jockey", "fashion designer", "comics artist", "dancer", "seiyū", "drummer", 
        "voice actor", "television producer", "designer", "visual artist", "chef", "beauty pageant contestant", 
        "playwright", "choreographer", "illustrator", "cinematographer", "cartoonist", "theatrical director", 
        "editor", "mangaka", "violinist", "television director", "film editor", "curator", "filmmaker", 
        "ballet dancer", "youtuber", "audio engineer", "pornographic actor", "graphic designer", "columnist", 
        "drag queen", "animator", "literary critic", "sports journalist", "director", "presenter", 
        "documentary filmmaker", "publisher", "children's writer", "science fiction writer", "make-up artist", 
        "non-fiction writer", "saxophonist", "costume designer", "contemporary artist", "blogger", "restaurateur", 
        "organist", "cellist", "bassist", "news presenter", "installation artist", "magician", "performance artist", 
        "motivational speaker", "video artist", "essayist", "announcer", "cook", "biographer", "film critic", 
        "trumpeter", "game designer", "stand-up comedian", "interior designer", "art collector", "art dealer", 
        "child actor", "exhibition curator", "clarinetist", "lyricist", "art critic", "printmaker", 
        "television personality", "entertainer", "percussionist", "keyboardist", "newspaper editor", 
        "photojournalist", "japanese idol", "vlogger", "podcaster", "comics writer", "socialite", "fiddler", 
        "penciller", "art director", "production designer", "puppeteer", "club dj", "autobiographer", 
        "classical guitarist", "fashion model", "bandleader", "reality television participant", 
        "multimedia artist", "music video director", "vocalist", "circus performer", "flautist", 
        "video game developer", "classical pianist", "jewelry designer", "textile artist", "caricaturist", 
        "glass artist", "banjoist", "lighting designer", "bass guitarist", "street artist", "weather presenter", 
        "talent agent", "owarai tarento", "opinion journalist", "board game designer", "potter", "music critic", 
        "film score composer", "scenographer", "radio producer", "influencer", "musical instrument maker"
    ],
    "STEM & Academia": [
        "physician", "scientist", "engineer", "academic", "computer scientist", "mathematician", "historian", 
        "economist", "researcher", "physicist", "university teacher", "psychologist", "architect", "chemist", 
        "biologist", "philosopher", "political scientist", "linguist", "sociologist", "anthropologist", "teacher", 
        "theologian", "translator", "astronomer", "art historian", "professor", "neuroscientist", "biochemist", 
        "archaeologist", "statistician", "botanist", "psychiatrist", "musicologist", "environmentalist", 
        "geneticist", "geologist", "electrical engineer", "epidemiologist", "astrophysicist", "geographer", 
        "ecologist", "civil engineer", "inventor", "librarian", "nurse", "social worker", "social scientist", 
        "explorer", "programmer", "zoologist", "paleontologist", "astronaut", "educator", "immunologist", 
        "mechanical engineer", "microbiologist", "meteorologist", "music educator", "literary scholar", 
        "academic administrator", "oncologist", "molecular biologist", "neurologist", "chemical engineer", 
        "pedagogue", "philologist", "pediatrician", "cardiologist", "ceramicist", "landscape architect", 
        "lecturer", "ophthalmologist", "virologist", "military historian", "classical scholar", 
        "historian of modern age", "entomologist", "criminologist", "oceanographer", "climatologist", 
        "veterinarian", "dentist", "materials scientist", "pharmacist", "psychotherapist", "biophysicist", 
        "gynecologist", "cryptographer", "pathologist", "geophysicist", "classical philologist", "archivist", 
        "neurosurgeon", "artificial intelligence researcher", "medical researcher", "biostatistician", 
        "literary historian", "religious studies scholar", "software developer", "conservationist", 
        "islamicist", "ornithologist", "biblical scholar", "pharmacologist", "physiologist", "marine biologist", 
        "theoretical physicist", "bioinformatician", "medievalist", "nutritionist", "herpetologist", "draftsperson", 
        "evolutionary biologist", "sinologist", "egyptologist"
    ],
    "Business": [
        "businessperson", "entrepreneur", "business executive", "banker", "chief executive officer", "manager", 
        "accountant", "music executive", "financier", "business theorist", "philanthropist", "consultant", 
        "manufacturer", "executive", "investment banker", "investor", "executive producer"
    ],
    "Military": [
        "military personnel", "military officer", "military leader", "naval officer", "military flight engineer", 
        "soldier", "army officer", "air force officer"
    ],
    "Religion": [
        "catholic priest", "anglican priest", "rabbi", "priest", "pastor", "missionary", "christian minister", 
        "eastern orthodox priest", "ʿālim", "imam"
    ],
    "Criminal": [
        "serial killer", "drug trafficker", "criminal", "terrorist"
    ],
    "Aviation": [
        "aircraft pilot"
    ],
    "Agriculture": [
        "farmer", "agronomist", "horticulturist", "winegrower"
    ]
}

# 2. Create a reverse mapping for efficient lookup.
occupation_to_bucket = {occ: bucket for bucket, occs in OCCUPATION_BUCKETS.items() for occ in occs}
        
# 3. Define a function to apply the mapping 
def bucket_occupation(occupation):
    # Strip whitespace from the input occupation to handle data inconsistencies
    clean_occupation = str(occupation).strip()
    return occupation_to_bucket.get(clean_occupation, 'Other')

# 4. Apply the function to create the new 'occupation_group' column.
print("Applying bucketing to the 'occupation' column...")
df['occupation_group'] = df['occupation'].apply(bucket_occupation)

# --- Verification ---
print("\n✅ Occupation bucketing complete.")
print("\nValue counts for the new 'occupation_group' column:")
bucket_counts = df['occupation_group'].value_counts()
bucket_percentages = df['occupation_group'].value_counts(normalize=True) * 100
summary_df = pd.DataFrame({
    'Count': bucket_counts,
    'Percentage': bucket_percentages.map('{:.2f}%'.format)
})
display(summary_df)

# --- Preview of 'Other' Category ---
print("\n--- Top 50 Occupations in the 'Other' Category ---")
print("This list shows the remaining occupations to be categorized.")

other_df = df[df['occupation_group'] == 'Other']

if other_df.empty:
    print("✅ No occupations fell into the 'Other' category. Bucketing is complete!")
else:
    # Get the value counts of the original occupations within the 'Other' group
    other_counts = other_df['occupation'].value_counts()
    display(other_counts.head(50))

Defining final occupation buckets...
Applying bucketing to the 'occupation' column...

✅ Occupation bucketing complete.

Value counts for the new 'occupation_group' column:


,Count,Percentage
occupation_group,,
Sports,513505,45.57%
Arts & Culture,270177,23.98%
Politics & Law,139232,12.36%
STEM & Academia,92524,8.21%
Other,78368,6.95%
Business,19162,1.70%
Military,7017,0.62%
Religion,5154,0.46%
Criminal,809,0.07%



--- Top 50 Occupations in the 'Other' Category ---
This list shows the remaining occupations to be categorized.


occupation
unknown                             51734
professional shogi player             238
software engineer                     102
music journalist                       89
bhikkhu                                70
dj producer                            70
violist                                69
dub actor                              69
co-driver                              69
short story writer                     69
talent manager                         69
nuclear physicist                      68
naturalist                             68
nun                                    68
historian of science                   67
artistic director                      67
orientalist                            67
solicitor                              66
stunt performer                        66
pentathlete                            66
music director                         66
visual effects supervisor              66
gridiron football player               65
industrial designer    

In [4]:
# Cell 4: Prepare for Time-Series Analysis

# This cell prepares our data for time-series analysis. 
# It extracts the creation year from the 'first_edit_ts' column and then filters the DataFrame to only include articles created since 2015, as specified in the project plan.

print("Extracting creation year from timestamps...")

# Create a new 'creation_year' column by accessing the .dt.year attribute
# of our datetime column.
df['creation_year'] = df['first_edit_ts'].dt.year

# --- Filter by Time Window ---
# The project plan specifies an analysis window from 2015 to the present.
# We'll filter the DataFrame to remove any articles created before 2015.

original_rows = len(df)
analysis_start_year = 2015

print(f"\nFiltering DataFrame to include years >= {analysis_start_year}...")

df_filtered = df[df['creation_year'] >= analysis_start_year].copy()

filtered_rows = len(df_filtered)
rows_removed = original_rows - filtered_rows

# --- Verification ---
print(f"\n✅ Filtering complete.")
print(f"Removed {rows_removed:,} rows created before {analysis_start_year}.")
print(f"Remaining rows for analysis: {filtered_rows:,}")

print("\nArticle counts per year in the filtered dataset:")
display(df_filtered['creation_year'].value_counts().sort_index())

Extracting creation year from timestamps...

Filtering DataFrame to include years >= 2015...

✅ Filtering complete.
Removed 589,935 rows created before 2015.
Remaining rows for analysis: 536,909

Article counts per year in the filtered dataset:


creation_year
2015.0    51419
2016.0    56588
2017.0    53673
2018.0    52532
2019.0    54959
2020.0    60366
2021.0    54803
2022.0    38749
2023.0    36881
2024.0    44191
2025.0    32748
Name: count, dtype: int64

In [5]:
# Cell 5: Create Yearly Aggregates

# This cell groups the data by year and by our three key dimensions and counts the number of biographies in each combination.

print("Aggregating data by year, gender, country, and occupation group...")

# Group the filtered DataFrame by our analysis columns and count the size of each group.
# .size() is efficient for just counting rows in groups.
# .reset_index(name='count') converts the resulting Series back into a DataFrame.
yearly_agg_df = (
    df_filtered.groupby([
        'creation_year',
        'gender',
        'country',
        'occupation_group'
    ])
    .size()
    .reset_index(name='count')
)

# --- Verification ---
print("\n✅ Aggregation complete.")
print(f"Created a summary table with {len(yearly_agg_df):,} unique group combinations.")

print("\nSample of the aggregated data (top rows):")
display(yearly_agg_df.head())

print("\nSample of the aggregated data (bottom rows):")
display(yearly_agg_df.tail())

Aggregating data by year, gender, country, and occupation group...

✅ Aggregation complete.
Created a summary table with 49,406 unique group combinations.

Sample of the aggregated data (top rows):


,creation_year,gender,country,occupation_group,count
0,2015.0,female,Afghanistan,Arts & Culture,6
1,2015.0,female,Afghanistan,Aviation,1
2,2015.0,female,Afghanistan,Politics & Law,6
3,2015.0,female,Afghanistan,STEM & Academia,1
4,2015.0,female,Afghanistan,Sports,1



Sample of the aggregated data (bottom rows):


,creation_year,gender,country,occupation_group,count
49401,2025.0,unknown,unknown,Arts & Culture,7
49402,2025.0,unknown,unknown,Business,2
49403,2025.0,unknown,unknown,Other,131
49404,2025.0,unknown,unknown,Politics & Law,3
49405,2025.0,unknown,unknown,STEM & Academia,8


In [6]:
# Cell 6: Final Filtering and Saving

# Filter out the rows where gender, country, AND occupation group are all 'unknown'

print(f"Original analysis rows: {len(df_filtered):,}")

# Keep rows that have at least ONE valid attribute for analysis
analysis_df = df_filtered[
    (df_filtered['gender'] != 'unknown') |
    (df_filtered['country'] != 'unknown') |
    (df_filtered['occupation_group'] != 'unknown')
].copy()

rows_removed = len(df_filtered) - len(analysis_df)
print(f"Removed {rows_removed:,} rows where all three attributes were 'unknown'.")
print(f"Final analysis rows: {len(analysis_df):,}")

# --- Re-aggregate the Cleaned Data ---
print("\nRe-aggregating the cleaned data...")
final_agg_df = (
    analysis_df.groupby([
        'creation_year',
        'gender',
        'country',
        'occupation_group'
    ])
    .size()
    .reset_index(name='count')
)

# --- Save the Final Aggregated Dataset ---
# This is the clean, summary data that will power our dashboard.
output_path = ROOT / "data" / "processed" / "yearly_aggregates.csv"
final_agg_df.to_csv(output_path, index=False)

print(f"\n✅ Final aggregated data saved to: {output_path.name}")
print("This notebook is now complete. The next step is visualization.")
display(final_agg_df.head())

Original analysis rows: 536,909
Removed 0 rows where all three attributes were 'unknown'.
Final analysis rows: 536,909

Re-aggregating the cleaned data...

✅ Final aggregated data saved to: yearly_aggregates.csv
This notebook is now complete. The next step is visualization.


,creation_year,gender,country,occupation_group,count
0,2015.0,female,Afghanistan,Arts & Culture,6
1,2015.0,female,Afghanistan,Aviation,1
2,2015.0,female,Afghanistan,Politics & Law,6
3,2015.0,female,Afghanistan,STEM & Academia,1
4,2015.0,female,Afghanistan,Sports,1
